# GatorTronS

## Train the model

In [1]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere

%run /home/ec2-user/SageMaker/LLM-NER-clinical-text/src/models/train_model.py \
--model_name 'UFNLP/gatortrons' \
--data_dir '/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/public/MedMentions/preprocessed-data/' \
--batch_size 8 \
--num_train_epochs 5 \
--learning_rate 5e-5 \
--weight_decay 0.01 \
--new_model_dir "/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/" \
--path_umls_semtype '/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/public/MedMentions/SemGroups_2018.txt'


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and preprocessing the dataset ...
/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/public/MedMentions/preprocessed-data/
The device to run the model: cuda
Load the pretrained model ...


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MegatronBertForTokenClassification were not initialized from the model checkpoint at UFNLP/gatortrons and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model has 354.262059millions parameters.


***** Running training *****
  Num examples = 2,635
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3,295
  Number of trainable parameters = 354,262,059


Epoch,Training Loss,Validation Loss,F1
1,0.524100,0.416685,0.603839
2,0.332700,0.401952,0.627280
3,0.228800,0.429635,0.633531
4,0.151200,0.470514,0.633126
5,0.103800,0.514751,0.633849


***** Running Evaluation *****
  Num examples = 878
  Batch size = 4
Saving model checkpoint to /home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/tmp-checkpoint-659
Configuration saved in /home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/tmp-checkpoint-659/config.json
Model weights saved in /home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/tmp-checkpoint-659/model.safetensors
tokenizer config file saved in /home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/tmp-checkpoint-659/tokenizer_config.json
Special tokens file saved in /home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/tmp-checkpoint-659/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 878
  Batch size = 4
Saving model checkpoint to /home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/tmp-checkpoint-1318
Configuration saved in /home/ec2-user/SageMaker/L

## Evaluate on test data


In [1]:
from src.models.model import *

# Load the model
model_loader = ModelLoader('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/')

# Evaluate the models on metrics
scores = model_loader.evaluate_model(dataset_name='../data/public/MedMentions/preprocessed-data/',\
                                     path_umls_semtype='../data/public/MedMentions/SemGroups_2018.txt',\
                                     metric_names=['f1', 'precision', 'recall', 'matthews_correlation'])

scores

/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and preprocessing the dataset ...
../data/public/MedMentions/preprocessed-data/


{'f1': 0.63101050849362,
 'precision': 0.6714479559316552,
 'recall': 0.6145681950682534,
 'matthews_correlation': 0.7203455851532575}

## Fix the id2label in config file

In [39]:
import json
from src.data.data_loader import *
from huggingface_hub import HfApi

# Load the config file
f = open('../models/medmentions/gatortrons/config.json')
config = json.load(f)

# Load the dataset with label mapping
dataset_loader = DatasetLoader(dataset_name='../data/public/MedMentions/preprocessed-data/', model_name='/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/', \
                               path_umls_semtype='../data/public/MedMentions/SemGroups_2018.txt')
dataset, classmap, umls_label_code, _ = dataset_loader.load_dataset()

# Add the label to the config
id_to_tag = {ind: classmap.int2str(ind) for ind in range(len(classmap.names))}
id_to_label = {ind: str(umls_label_code[id_to_tag[ind]]) for ind in range(len(classmap.names))}
config['id2label'] = id_to_label
config['label2id'] = {id_to_label[key]: key for key in id_to_label.keys()}
# Save the config
with open('../models/medmentions/gatortrons/config.json', 'w') as f:
    json.dump(config, f)
    
# Push the config to hub
api = HfApi()
api.upload_file(
    path_or_fileobj="../models/medmentions/gatortrons/config.json",
    path_in_repo="config.json",
    repo_id="longluu/Clinical-NER-MedMentions-GatorTronS",
    repo_type="model",
    commit_message="fix the label2id in config"
)

Loading and preprocessing the dataset ...
../data/public/MedMentions/preprocessed-data/


CommitInfo(commit_url='https://huggingface.co/longluu/Clinical-NER-MedMentions-GatorTronS/commit/357717279c7386e6e7b4d0203186085ed88d5a13', commit_message='fix the label2id in config', commit_description='', oid='357717279c7386e6e7b4d0203186085ed88d5a13', pr_url=None, pr_revision=None, pr_num=None)

## Push to model to hub

In [30]:
from huggingface_hub import notebook_login

notebook_login()


In [47]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer

# Push the model to hub
NER_model.push_to_hub("longluu/Clinical-NER-MedMentions-GatorTronS", commit_message='--batch_size 4 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01')
tokenizer.push_to_hub("longluu/Clinical-NER-MedMentions-GatorTronS", commit_message='--batch_size 4 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01')

README.md: 100%|██████████| 21.0/21.0 [00:00<00:00, 3.65kB/s]
model.safetensors: 100%|██████████| 1.42G/1.42G [00:34<00:00, 41.3MB/s]


CommitInfo(commit_url='https://huggingface.co/longluu/Clinical-NER-MedMentions-GatorTronS/commit/e6a951cf15bd341dacaf6c242100e784c7f0dee3', commit_message='--batch_size 4 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01', commit_description='', oid='e6a951cf15bd341dacaf6c242100e784c7f0dee3', pr_url=None, pr_revision=None, pr_num=None)

# GatorTron-base

## Train the model

In [1]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere

%run /home/ec2-user/SageMaker/LLM-NER-clinical-text/src/models/train_model.py \
--model_name 'UFNLP/gatortron-base' \
--data_dir '/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/public/MedMentions/preprocessed-data/' \
--batch_size 4 \
--num_train_epochs 5 \
--learning_rate 5e-5 \
--weight_decay 0.01 \
--new_model_dir "/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortron-base/" \
--path_umls_semtype '/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/public/MedMentions/SemGroups_2018.txt'


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and preprocessing the dataset ...
/home/ec2-user/SageMaker/LLM-NER-clinical-text/data/public/MedMentions/preprocessed-data/
The device to run the model: cuda
Load the pretrained model ...


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MegatronBertForTokenClassification were not initialized from the model checkpoint at UFNLP/gatortron-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model has 354.262059millions parameters.


Epoch,Training Loss,Validation Loss,F1
1,0.553000,0.427210,0.595944
2,0.350800,0.411381,0.619327
3,0.251200,0.425950,0.626551
4,0.176400,0.460033,0.628520
5,0.128000,0.497196,0.629384


## Evaluate on test data


In [1]:
from src.models.model import *

# Load the model
model_loader = ModelLoader('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortron-base/')

# Evaluate the models on metrics
scores = model_loader.evaluate_model(dataset_name='../data/public/MedMentions/preprocessed-data/',\
                                     path_umls_semtype='../data/public/MedMentions/SemGroups_2018.txt',\
                                     metric_names=['f1', 'precision', 'recall', 'matthews_correlation'])

scores

/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and preprocessing the dataset ...
../data/public/MedMentions/preprocessed-data/


Generating train split: 0 examples [00:00, ? examples/s]/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 2635 examples [00:00, 10628.05 examples/s]
Generating validation split: 0 examples [00:00, ? examples/s]/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating validation split: 878 examples [00:00, 10999.40 examples/s]
Generating test split: 0 examples [

{'f1': 0.6271402249699903,
 'precision': 0.6691625224055963,
 'recall': 0.6085333637974402,
 'matthews_correlation': 0.720898121696139}

## Fix the id2label in config file

In [41]:
import json
from src.data.data_loader import *
from huggingface_hub import HfApi

# Load the config file
f = open('../models/medmentions/gatortrons/config.json')
config = json.load(f)

# Load the dataset with label mapping
dataset_loader = DatasetLoader(dataset_name='../data/public/MedMentions/preprocessed-data/', model_name='/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortrons/', \
                               path_umls_semtype='../data/public/MedMentions/SemGroups_2018.txt')
dataset, classmap, umls_label_code, _ = dataset_loader.load_dataset()

# Add the label to the config
id_to_tag = {ind: classmap.int2str(ind) for ind in range(len(classmap.names))}
id_to_label = {ind: str(umls_label_code[id_to_tag[ind]]) for ind in range(len(classmap.names))}
config['id2label'] = id_to_label
config['label2id'] = {id_to_label[key]: key for key in id_to_label.keys()}
# Save the config
with open('../models/medmentions/gatortron-base/config.json', 'w') as f:
    json.dump(config, f)
    

Loading and preprocessing the dataset ...
../data/public/MedMentions/preprocessed-data/


## Push to model to hub

In [30]:
from huggingface_hub import notebook_login

notebook_login()


In [42]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer

# Load the model
tokenizer = AutoTokenizer.from_pretrained('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortron-base/')
NER_model = AutoModelForTokenClassification.from_pretrained('/home/ec2-user/SageMaker/LLM-NER-clinical-text/models/medmentions/gatortron-base/')
                                                            
# Push the model to hub
NER_model.push_to_hub("longluu/Clinical-NER-MedMentions-GatorTronBase", commit_message='--batch_size 4 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01')
tokenizer.push_to_hub("longluu/Clinical-NER-MedMentions-GatorTronBase", commit_message='--batch_size 4 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01')

README.md: 100%|██████████| 21.0/21.0 [00:00<00:00, 3.52kB/s]
model.safetensors: 100%|██████████| 1.42G/1.42G [00:34<00:00, 41.1MB/s]


CommitInfo(commit_url='https://huggingface.co/longluu/Clinical-NER-MedMentions-GatorTronBase/commit/0d3d619bf64c19649f358ab0688c26e22c51edfa', commit_message='--batch_size 4 --num_train_epochs 5 --learning_rate 5e-5 --weight_decay 0.01', commit_description='', oid='0d3d619bf64c19649f358ab0688c26e22c51edfa', pr_url=None, pr_revision=None, pr_num=None)